# Converting to standard form -- a step-by-step example

Convert the following problem to standard form:

$\begin{align}
\min_{x_1,x_2} & \ x_1 + x_2\\
\text{s.t.} & \ 5𝑥_1−3𝑥_2 = 7\\
& 2𝑥_1+𝑥_2≥2\\
& 1≤𝑥_2≤4\\
& 𝑥_1 \ \text{free}
\end{align}$

A good way to check that you've done it correctly is by solving both the original and the standard form models in Julia. If your model is correct, you should get the same answer.

## Original Model

In [1]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, x1) # x1 is free
@variable(m, 1 <= x2 <= 4 ) # x2 is bounded above and below

# an inequality constraint and an equality constraint
@constraint(m, 5*x1 - 3*x2 == 7 ) 
@constraint(m, 2*x1 + x2 >= 2 )

# objective is to minimize x1 + x2
@objective(m, Min, x1 + x2 )

# solve the problem
optimize!(m)

# display the model
println(m)
println()
println("x1 = ", value(x1) )
println("x2 = ", value(x2) )
println("objective = ", objective_value(m) )

Min x1 + x2
Subject to
 5 x1 - 3 x2 == 7.0
 2 x1 + x2 >= 2.0
 x2 >= 1.0
 x2 <= 4.0


x1 = 2.0
x2 = 1.0
objective = 3.0
Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 3
Coin0511I After Postsolve, objective 3, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 3 - 0 iterations time 0.052, Presolve 0.05


## Converting to Standard Form

When we convert it, the model should look like this:
<p style="text-align: center;">
$\begin{align}
\underset{x}{\max} \ & c^Tx & \\
\text{s.t.} \ & Ax \leq b &\\
& x \geq 0 &
\end{align}$
    
* We bound $x_1$ by replacing it with $x_1 = u - v, \ u, v \geq 0$:

<p style="text-align: center;">
$\begin{align}
\underset{u,v,x_2}{\min} \ & (u-v) + x_2 & \\
\text{s.t.} \ & 5(u-v) - 3x_2 = 7 &\\
& 2(u-v) + x_2 \geq 2 &\\
& 1 \leq x_2 \leq 4 & \\
& u, v \geq 0&
\end{align}$
</p>

* We shift the bounds on $x_2$ by using $w = x_2 - 1$:

<p style="text-align: center;">
$\begin{align}
\underset{u,v,w}{\min} \ & (u-v) + (w+1) & \\
\text{s.t.} \ & 5(u-v) - 3(w+1) = 7 &\\
& 2(u-v) + (w+1) \geq 2 &\\
& (w+1) \leq 4 & \\
& u, v, w \geq 0&
\end{align}$
</p>

* Rearrange so all constants are on the right-hand-side:

<p style="text-align: center;">
$\begin{align}
\underset{u,v,w}{\min} \ & u - v + w + 1 & \\
\text{s.t.} \ & 5u- 5v - 3w = 10 &\\
& 2u - 2v + w \geq 1 &\\
& w \leq 3 & \\
& u, v, w \geq 0&
\end{align}$
</p>

* We turn the min into a max:
<p style="text-align: center;">
$\begin{align}
\underset{u,v,w}{-\max}\ & -u + v - w - 1 & \\
\text{s.t.} \ & 5u- 5v - 3w = 10 &\\
& 2u - 2v + w \geq 1 &\\
& w \leq 3 & \\
& u, v, w \geq 0&
\end{align}$
</p>

* Flip $\geq$ inequality and replace equality with two inequalities:
<p style="text-align: center;">
$\begin{align}
\underset{u,v,w}{-\max}\ & -u + v - w - 1 & \\
\text{s.t.} \ & 5u- 5v - 3w \leq 10 &\\
& 5u- 5v - 3w \geq 10 &\\
& -2u + 2v - w \leq -1 &\\
& w \leq 3 & \\
& u, v, w \geq 0&
\end{align}$
</p>

* Finally, flip the newly created $\geq$ inequality and we're done!
<p style="text-align: center;">
$\begin{align}
\underset{u,v,w}{-\max}\ & -u + v - w - 1 & \\
\text{s.t.} \ & 5u- 5v - 3w \leq 10 &\\
& -5u + 5v + 3w \leq -10 &\\
& -2u + 2v - w \leq -1 &\\
& w \leq 3 & \\
& u, v, w \geq 0&
\end{align}$
</p>

Verify correctness by solving the new model in Julia:

## Standard Form Model

In [2]:
using JuMP, Clp

m = Model(Clp.Optimizer)

# three nonnegative variables
@variable(m, u >= 0 )
@variable(m, v >= 0 )
@variable(m, w >= 0 )

# four less than or equal to constraints
@constraint(m, 5*u - 5*v - 3*w <= 10 )
@constraint(m, -5*u + 5*v + 3*w <= -10 )
@constraint(m, -2*u + 2*v - w <= -1 )
@constraint(m, w <= 3 )

# maximize the objective
@objective(m, Max, -u + v - w - 1 )

# solve the standard form model
optimize!(m)

# display the model and solution
println(m)
println()
# remember to convert back to x1 and x2!
println("x1 = ", value(u-v) )
println("x2 = ", value(w+1) )
# remember min f(x) = -max -f(x), so we need to report the negative of our objective value
println("objective = ", -objective_value(m) )

Max -u + v - w - 1
Subject to
 5 u - 5 v - 3 w <= 10.0
 -5 u + 5 v + 3 w <= -10.0
 -2 u + 2 v - w <= -1.0
 w <= 3.0
 u >= 0.0
 v >= 0.0
 w >= 0.0


x1 = 2.0
x2 = 1.0
objective = 3.0
Coin0506I Presolve 3 (-1) rows, 3 (0) columns and 9 (-1) elements
Clp0006I 0  Obj -2.9 Primal inf 0.099999 (1) Dual inf 0.999999 (1)
Clp0006I 1  Obj -3
Clp0000I Optimal - objective value -3
Coin0511I After Postsolve, objective -3, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -3 - 1 iterations time 0.012, Presolve 0.00
